# Transfer learning with VGG16

In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7541595806790781323
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11297803469
locality {
  bus_id: 1
}
incarnation: 7535197879540138320
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
ls

In [0]:
!wget train-images-idx3-ubyte.gz

--2018-03-09 08:19:34--  https://github.com/avinashselvam/datasets/raw/master/data.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/avinashselvam/datasets/master/data.zip [following]
--2018-03-09 08:19:34--  https://raw.githubusercontent.com/avinashselvam/datasets/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6450075 (6.2M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   6.15M  14.8MB/s    in 0.4s    

2018-03-09 08:19:35 (14.8 MB/s) - ‘data.zip’ saved [6450075/6450075]



In [0]:
 !mkdir data1 && unzip data.zip -d catsdogs/

Archive:  data.zip
   creating: catsdogs/data/
   creating: catsdogs/data/valid/
   creating: catsdogs/data/valid/dogs/
  inflating: catsdogs/data/valid/dogs/dog.1035.jpg  
   creating: catsdogs/__MACOSX/
   creating: catsdogs/__MACOSX/data/
   creating: catsdogs/__MACOSX/data/valid/
   creating: catsdogs/__MACOSX/data/valid/dogs/
  inflating: catsdogs/__MACOSX/data/valid/dogs/._dog.1035.jpg  
  inflating: catsdogs/data/valid/dogs/dog.1021.jpg  
  inflating: catsdogs/__MACOSX/data/valid/dogs/._dog.1021.jpg  
  inflating: catsdogs/data/valid/dogs/dog.1009.jpg  
  inflating: catsdogs/__MACOSX/data/valid/dogs/._dog.1009.jpg  
  inflating: catsdogs/data/valid/dogs/dog.1008.jpg  
  inflating: catsdogs/__MACOSX/data/valid/dogs/._dog.1008.jpg  
  inflating: catsdogs/data/valid/dogs/dog.1020.jpg  
  inflating: catsdogs/__MACOSX/data/valid/dogs/._dog.1020.jpg  
  inflating: catsdogs/data/valid/dogs/dog.1034.jpg  
  inflating: catsdogs/__MACOSX/data/valid/dogs/._dog.1034.jpg  
  inflating: catsd

In [0]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [0]:
!cd data/__MACOSX/data/train/dogs && ls

In [0]:
PATH = 'catsdogs/data/'

In [0]:
!ls {PATH}/valid/cats | head

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

58900480/58889256 [==============================] - 1s 0us/step


In [0]:
model = Model(inputs=base_model.input,
              outputs=base_model.output)

In [0]:
datagen = ImageDataGenerator(rescale=1./255)

In [0]:
batch_size = 16

generator = datagen.flow_from_directory(PATH+'/train',
                                        target_size=(150, 150),
                                        batch_size=batch_size,
                                        shuffle=False)

Found 200 images belonging to 2 classes.


In [0]:
bottleneck_features_train = model.predict_generator(generator)

In [0]:
np.save(open('bottleneck_features_train.npy', 'w'),
        bottleneck_features_train)

In [0]:
generator = datagen.flow_from_directory(PATH+'/valid',
                                        target_size=(150, 150),
                                        batch_size=batch_size,
                                        shuffle=False)

Found 80 images belonging to 2 classes.


In [0]:
bottleneck_features_validation = model.predict_generator(generator)

In [0]:
np.save(open('bottleneck_features_validation.npy', 'w'),
        bottleneck_features_validation)

In [0]:
train_data = np.load(open('bottleneck_features_train.npy'))
train_labels = np.array([0]*100 + [1]*100)

In [0]:
validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0]*40 + [1]*40)

In [0]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [0]:
model.fit(train_data, train_labels,
         epochs=50,
         batch_size=batch_size,
         validation_data=(validation_data, validation_labels))

model.save_weights('bottleneck_fc_model.h5')

Train on 200 samples, validate on 80 samples
Epoch 1/50
200/200 [==============================] - 0s 2ms/step - loss: 5.9221 - acc: 0.5050 - val_loss: 0.5645 - val_acc: 0.6500
Epoch 2/50
200/200 [==============================] - 0s 463us/step - loss: 1.3018 - acc: 0.6000 - val_loss: 0.5406 - val_acc: 0.7375
Epoch 3/50
200/200 [==============================] - 0s 470us/step - loss: 0.4465 - acc: 0.8350 - val_loss: 0.6086 - val_acc: 0.7000
Epoch 4/50
200/200 [==============================] - 0s 470us/step - loss: 0.4190 - acc: 0.8500 - val_loss: 0.3128 - val_acc: 0.8500
Epoch 5/50
200/200 [==============================] - 0s 474us/step - loss: 0.3536 - acc: 0.8700 - val_loss: 0.4552 - val_acc: 0.8000
Epoch 6/50
200/200 [==============================] - 0s 517us/step - loss: 0.2611 - acc: 0.8800 - val_loss: 0.2524 - val_acc: 0.8750
Epoch 7/50
200/200 [==============================] - 0s 468us/step - loss: 0.1172 - acc: 0.9500 - val_loss: 0.4596 - val_acc: 0.8125
Epoch 8/50
200/200 

Epoch 22/50
200/200 [==============================] - 0s 527us/step - loss: 0.0681 - acc: 0.9850 - val_loss: 0.4117 - val_acc: 0.8875
Epoch 23/50
200/200 [==============================] - 0s 516us/step - loss: 0.0059 - acc: 0.9950 - val_loss: 0.3705 - val_acc: 0.9250
Epoch 24/50
200/200 [==============================] - 0s 515us/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.3604 - val_acc: 0.9125
Epoch 25/50
200/200 [==============================] - 0s 512us/step - loss: 9.3003e-04 - acc: 1.0000 - val_loss: 0.3946 - val_acc: 0.9125
Epoch 26/50
200/200 [==============================] - 0s 520us/step - loss: 0.0341 - acc: 0.9750 - val_loss: 0.3768 - val_acc: 0.9250
Epoch 27/50
200/200 [==============================] - 0s 505us/step - loss: 4.5645e-04 - acc: 1.0000 - val_loss: 0.3506 - val_acc: 0.9250
Epoch 28/50
200/200 [==============================] - 0s 527us/step - loss: 0.0056 - acc: 1.0000 - val_loss: 0.3656 - val_acc: 0.9250
Epoch 29/50
200/200 [==========================

200/200 [==============================] - 0s 528us/step - loss: 5.3332e-05 - acc: 1.0000 - val_loss: 0.3875 - val_acc: 0.9375
Epoch 44/50
200/200 [==============================] - 0s 507us/step - loss: 1.7601e-04 - acc: 1.0000 - val_loss: 0.3818 - val_acc: 0.9250
Epoch 45/50
200/200 [==============================] - 0s 537us/step - loss: 4.3025e-05 - acc: 1.0000 - val_loss: 0.3745 - val_acc: 0.9375
Epoch 46/50
200/200 [==============================] - 0s 493us/step - loss: 3.1456e-05 - acc: 1.0000 - val_loss: 0.3851 - val_acc: 0.9375
Epoch 47/50
200/200 [==============================] - 0s 514us/step - loss: 4.3419e-06 - acc: 1.0000 - val_loss: 0.3906 - val_acc: 0.9375
Epoch 48/50
200/200 [==============================] - 0s 532us/step - loss: 9.7429e-05 - acc: 1.0000 - val_loss: 1.3988 - val_acc: 0.7875
Epoch 49/50
200/200 [==============================] - 0s 528us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.6916 - val_acc: 0.8625
Epoch 50/50
200/200 [======================

In [0]:
from google.colab import files
files.download('bottleneck_fc_model.h5')